In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#opening connection & cursor
conn = psycopg2.connect(
                            host="lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com",
                            database="mid_term_project",
                            user="lhl_student",
                            password="lhl_student")
cursor = conn.cursor()

def create_pandas_table(sql_query, database = conn):
    table = pd.read_sql_query(sql_query, database)
    return table

In [66]:

# for i in range(1, 13):
#     if i == 1:
#         test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND WHERE fl_date IN ('2018-01-01','2018-01-02','2018-01-03','2018-01-04','2018-01-05','2018-01-06','2018-01-07','2019-01-01','2019-01-02','2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07');"
#         df = create_pandas_table(test2)
#     else:
#         test2 = f"SELECT * FROM flights WHERE arr_delay IS NOT NULL AND weather_delay IS NOT NULL AND arr_delay < 200 AND EXTRACT(month from CAST(fl_date as date)) = {i} AND EXTRACT(year from CAST(fl_date as date)) = 2019 LIMIT 100000;"
#         test_pd_table = create_pandas_table(test2)
#         df = pd.concat([df, test_pd_table])  ### AND weather_delay IS NOT NULL 
test2 = "SELECT * FROM flights WHERE arr_delay IS NOT NULL AND arr_delay < 200 AND fl_date IN ('2018-01-01','2018-01-02','2018-01-03','2018-01-04','2018-01-05','2018-01-06','2018-01-07','2019-01-01','2019-01-02','2019-01-03','2019-01-04','2019-01-05','2019-01-06','2019-01-07');"
df = create_pandas_table(test2)

In [67]:
df.shape

(275712, 42)

In [24]:
df.fl_date.unique()

array(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
       '2019-01-05', '2019-01-06', '2019-01-07', '2018-01-01',
       '2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05',
       '2018-01-06', '2018-01-07'], dtype=object)

In [25]:
passengers = create_pandas_table("SELECT * FROM passengers TABLESAMPLE SYSTEM(10) WHERE year >= 2019 ")

In [26]:
passengers.shape

(50068, 38)

In [27]:
passengers.passengers.max()

68773.0

In [28]:
fuel = create_pandas_table("SELECT * FROM fuel_comsumption WHERE year >= 2019 ")

In [29]:
fuel.shape
fuel.total_gallons

0        1235210.0
1              0.0
2          60821.0
3              0.0
4         410186.0
          ...     
453      3943414.0
454            0.0
455            0.0
456        22895.0
457    162505172.0
Name: total_gallons, Length: 458, dtype: float64

In [30]:
fuel_mean=fuel[['unique_carrier','total_gallons']].groupby(['unique_carrier'],as_index=False).mean()

In [31]:
fuel_mean.rename(columns={'unique_carrier':'op_unique_carrier'},inplace=True)


In [32]:
passengers['capacity'] = passengers['seats'] - passengers['passengers']


In [33]:
passengers.rename(columns={'unique_carrier':'mkt_unique_carrier'},inplace=True)


In [34]:
passengers_mean=passengers[['mkt_unique_carrier','origin','dest','passengers', 'capacity']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [35]:
passengers_mean.head()

,mkt_unique_carrier,origin,dest,passengers,capacity
0,02Q,CMH,EDI,265.0,0.0
1,02Q,CMH,STN,254.0,11.0
2,02Q,EDI,CMH,261.0,4.0
3,02Q,LAX,EDI,19.0,13.0
4,02Q,LAX,YHZ,19.0,13.0


In [36]:
df_merged=pd.merge(df,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [37]:
df_merged.shape

(198500, 44)

In [38]:
df_merged=pd.merge(df_merged,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [39]:
df_merged.shape

(195374, 45)

In [40]:
df_merged.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                  69
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      101817
diverted                    0
dup                         0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [41]:
df_merged.dropna(axis=0, subset=['dep_delay'], inplace=True)

In [42]:
# remember to close connection to allow for others to use the bandwidth
cursor.close()
conn.close()

In [43]:
df_merged.passengers.unique()

array([1325.4       ,  126.        , 8486.33333333, ..., 1292.        ,
       1324.33333333, 1167.33333333])

In [44]:
df.mkt_unique_carrier.unique()

array(['AA', 'NK', 'AS', 'DL', 'UA', 'WN', 'G4', 'HA', 'B6', 'F9', 'VX'],
      dtype=object)

In [45]:
df.shape

(275712, 42)

In [46]:
df_merged.shape

(191745, 41)

In [47]:
df_merged.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,passengers,capacity,total_gallons
0,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,NaN,NaN,NaN,None,NaN,NaN,None,1325.4,157.0,0.0
1,2019-01-01,AA,AA_CODESHARE,AA,5243,OH,N591NN,5243,13577,MYR,...,NaN,NaN,NaN,None,NaN,NaN,None,1325.4,157.0,0.0
2,2019-01-01,AA,AA_CODESHARE,AA,5249,OH,N578NN,5249,13577,MYR,...,NaN,NaN,NaN,None,NaN,NaN,None,1325.4,157.0,0.0
3,2019-01-01,AA,AA_CODESHARE,AA,5261,OH,N584NN,5261,13577,MYR,...,NaN,NaN,NaN,None,NaN,NaN,None,1325.4,157.0,0.0
4,2019-01-01,AA,AA_CODESHARE,AA,5358,OH,N577NN,5358,13577,MYR,...,NaN,NaN,NaN,None,NaN,NaN,None,1325.4,157.0,0.0


In [48]:
df_test = df_merged[['mkt_unique_carrier','origin','dest','dep_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'weather_delay', 'carrier_delay']].groupby(['mkt_unique_carrier','origin','dest', ],as_index=False).mean()

In [49]:
df_merged.drop(columns=['first_dep_time','total_add_gtime','longest_add_gtime', 'no_name', 'dep_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay'], inplace=True)

In [50]:
df_merged = pd.merge(df_merged, df_test, on=['mkt_unique_carrier', 'origin', 'dest'] )

In [69]:
# df_merged = df_merged.drop(columns=['dep_delay_x', 'carrier_delay_x', 'weather_delay_x', 'nas_delay_x', 'security_delay_x',
#        'late_aircraft_delay_x',])

In [91]:
len(df_merged.columns)

41

In [ ]:
df_merged[]

In [52]:
df_merged.isnull().sum()

fl_date                    0
mkt_unique_carrier         0
branded_code_share         0
mkt_carrier                0
mkt_carrier_fl_num         0
op_unique_carrier          0
tail_num                   0
op_carrier_fl_num          0
origin_airport_id          0
origin                     0
origin_city_name           0
dest_airport_id            0
dest                       0
dest_city_name             0
crs_dep_time               0
dep_time                   0
taxi_out                   0
wheels_off                 0
wheels_on                  0
taxi_in                    0
crs_arr_time               0
arr_time                   0
arr_delay                  0
cancelled                  0
cancellation_code      99613
diverted                   0
dup                        0
crs_elapsed_time           0
actual_elapsed_time        0
air_time                   0
flights                    0
distance                   0
passengers                 0
capacity                   0
total_gallons 

In [57]:
flights_test = create_pandas_table("SELECT * FROM flights_test WHERE fl_date IN ('2020-01-01','2020-01-02','2020-01-03','2020-01-04','2020-01-05','2020-01-06','2020-01-07')")

In [58]:
flights_test.shape

(150623, 20)

In [59]:
flights_test=pd.merge(flights_test,passengers_mean,on=['mkt_unique_carrier', 'origin','dest'], how='inner') 

In [60]:
flights_test=pd.merge(flights_test,fuel_mean, on=['op_unique_carrier'], how='inner') 

In [61]:
flights_test.shape

(101569, 23)

In [13]:
test3 = "SELECT * FROM flights TABLESAMPLE SYSTEM(4) WHERE arr_delay IS NOT NULL AND arr_delay < 200 "
new_df = create_pandas_table(test3)

In [20]:
new_df.shape

(115349, 42)

In [18]:
new_df.dropna(axis=0, subset=['taxi_out', 'weather_delay', 'wheels_on', 'dep_delay'], inplace=True)

In [19]:
new_df.isna().sum()

fl_date                     0
mkt_unique_carrier          0
branded_code_share          0
mkt_carrier                 0
mkt_carrier_fl_num          0
op_unique_carrier           0
tail_num                    0
op_carrier_fl_num           0
origin_airport_id           0
origin                      0
origin_city_name            0
dest_airport_id             0
dest                        0
dest_city_name              0
crs_dep_time                0
dep_time                    0
dep_delay                   0
taxi_out                    0
wheels_off                  0
wheels_on                   0
taxi_in                     0
crs_arr_time                0
arr_time                    0
arr_delay                   0
cancelled                   0
cancellation_code      111178
diverted                    0
dup                         0
crs_elapsed_time            0
actual_elapsed_time         0
air_time                    0
flights                     0
distance                    0
carrier_de

In [62]:
flights_test.to_csv('flights_test_grouped.csv')

In [ ]:
df_merged.to_csv('flights_merged.csv')

In [21]:
new_df.to_csv('flights_sample.csv')